# Use pygif and Open Plant Hardiness Zones (OPHZ) to add hardiness zone to GBIF data

In [1]:
# imports
from pygbif import occurrences as occ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# removes pandas output limit
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Query the GBIF API (this is where you can edit your query)

In [5]:
# Creates df list (DO NOT RERUN BETWEEN QUERIES)
occdfs_list = []

In [6]:
# Max records per search is 300, can be improved using download instead
records = 300*100
prev_offset = 0 # update on repeated queries to build a dataframe

In [7]:
# Fetches data from GBIF API in 300 record chunks and adds each chunk to a list
for offset in range(0,records,300):
    occdfs_list.append(pd.DataFrame(occ.search(
        country='US',
        has_coordinate=True,
        hasGeospatialIssue = False,
        return_type='dataframe',
        limit=300,
        offset=prev_offset+offset,
    )['results']))

KeyboardInterrupt: 

In [9]:
# Concats list of dataframes into one dataframe
occdf_full = pd.concat(occdfs_list)

# Creates new dataframe with only the columns we want
occdf = occdf_full[["datasetKey", "decimalLatitude", "decimalLongitude", "scientificName", "taxonKey"]]

### Combine OPHZ geometries for each state into one index

In [10]:
import geopandas as gpd
import os

# Get a list of all GeoJSON files in the folder
file_list = [f for f in os.listdir('./ophz/geojson/') if f.endswith('.geojson')]

# Initialize an empty list to store the GeoDataFrames
gdf_list = []

# Iterate through the list of files
for file in file_list:
    # Read in the GeoJSON file
    gdf_state = gpd.read_file(os.path.join('./ophz/geojson/', file))
    # Append the GeoDataFrame to the list
    gdf_list.append(gdf_state)

# Concatenate the list of GeoDataFrames into a single GeoDataFrame
gdf = pd.concat(gdf_list)

# Create a spatial index for the GeoDataFrame
sindex = gdf.sindex

### Combine OPHZ with GBIF data

In [12]:
# supresses shapely warnings
import warnings
warnings.filterwarnings('ignore')

from shapely.geometry import Point

# Initialize an empty list to store the matching features
matching_features = []

# Loops through each occurence in the occurrence dataframe
for i in range(len(occdf)):
    # Finds potential matches
    lon = occdf.iloc[i].decimalLongitude
    lat = occdf.iloc[i].decimalLatitude
    potential_matches = list(sindex.intersection((lon, lat)))
    # Loops through each potential match
    for j in potential_matches:
        # Checks if the occurence is within the polygon
        try:
            if gdf.iloc[j].geometry.contains(Point(lon, lat)):
                # Combine occurence entry with matching features using pandas.concat
                matching_features.append(pd.concat([occdf.iloc[i], gdf.iloc[j][["TEMP", "ZONE"]]]))
        except:
            pass
        

ophz_gbif_df = pd.DataFrame(matching_features)
# matching_df

ERROR:TopologyException: side location conflict at -115.18578421052631 47.129247368421048. This can occur if the input geometry is invalid.
INFO:Ring Self-intersection at or near point -115.88209999999999 48.701999999999998
ERROR:TopologyException: side location conflict at -115.18578421052631 47.129247368421048. This can occur if the input geometry is invalid.
INFO:Ring Self-intersection at or near point -115.88209999999999 48.701999999999998
ERROR:TopologyException: side location conflict at -115.18578421052631 47.129247368421048. This can occur if the input geometry is invalid.
INFO:Ring Self-intersection at or near point -115.88209999999999 48.701999999999998
ERROR:TopologyException: side location conflict at -115.18578421052631 47.129247368421048. This can occur if the input geometry is invalid.
INFO:Ring Self-intersection at or near point -115.88209999999999 48.701999999999998
ERROR:TopologyException: side location conflict at -115.18578421052631 47.129247368421048. This can occu

# Saves the data to a CSV file (includes datasetKey for easy reupload to GBIF)

In [13]:
ophz_gbif_df.to_csv('ophz_gbif.csv')

## Map everything using Folium

In [14]:
# Displays results on a map with temp as the color
# Overlays it on a map of the US, uses lat and lon to place each occurence
import matplotlib.pyplot as plt
import matplotlib
import folium
from folium.plugins import FastMarkerCluster


# Maps temp to colors, -40 is pink, +40 is red, 0 is green
cmap = plt.cm.get_cmap('RdYlGn')
norm = plt.Normalize(vmin=-40, vmax=40)

folium_map = folium.Map(location=[38, -97],
                        zoom_start=4.4)

# These two lines should create FastMarkerClusters
FastMarkerCluster(data=list(zip(ophz_gbif_df['decimalLatitude'].values, ophz_gbif_df['decimalLongitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)

for index, row in ophz_gbif_df.iterrows():
   # generate the popup message that is shown on click.
   popup_text = "{}<br> Zone: {}<br> Temp: {}"
   popup_text = popup_text.format(
                     row["scientificName"],
                     row["ZONE"],
                     row["TEMP"]
                     )
   # add marker to the map
   folium.CircleMarker(location=(row['decimalLatitude'],
                                 row['decimalLongitude']),
                     radius= 3,
                     color=matplotlib.colors.to_hex(cmap(norm(row['TEMP']))),
                     popup=popup_text).add_to(folium_map)

# displays map
folium_map.save("map.html")